**Table of contents**<a id='toc0_'></a>    
- [Initialize TSDataset, Pipeline, Model, Validator, Strategy](#toc1_1_)    
    - [TSDataset](#toc1_1_1_)    
    - [Pipeline](#toc1_1_2_)    
    - [Trainer](#toc1_1_3_)    
    - [Strategy](#toc1_1_4_)    
- [Save and load checkpoints](#toc2_)    
    - [Save checkpoint](#toc2_1_1_)    
  - [Load checkpoint for finetune](#toc2_2_)    
  - [Load checkpoint for inference](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import warnings

warnings.filterwarnings("ignore")

from pathlib import Path
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import torch

from tsururu.dataset import Pipeline, TSDataset
from tsururu.model_training.trainer import DLTrainer
from tsururu.model_training.validator import KFoldCrossValidator
from tsururu.models.torch_based.dlinear import DLinear_NN
from tsururu.strategies import MIMOStrategy
from tsururu.transformers import (
    DateSeasonsGenerator,
    LagTransformer,
    SequentialTransformer,
    TargetGenerator,
    UnionTransformer,
)

In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(value, idx_fold)
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

## <a id='toc1_1_'></a>[Initialize TSDataset, Pipeline, Model, Validator, Strategy](#toc0_)

The initialization of the main components is exactly the same as when using ML models. The only difference is that `DLTrainer` allows you to pass many more parameters compared to `MLTrainer`.

### <a id='toc1_1_1_'></a>[TSDataset](#toc0_)

In [ ]:
df_path = Path("../datasets/global/simulated_data_to_check.csv")

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continuous",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [4]:
dataset = TSDataset(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)

freq: Day; period: 1


### <a id='toc1_1_2_'></a>[Pipeline](#toc0_)

In [5]:
lag = LagTransformer(lags=7)
target_generator = TargetGenerator()
date_features = DateSeasonsGenerator()
date_lags = LagTransformer(lags=7)

union_1 = UnionTransformer(transformers_list=[lag, target_generator])
seq_1 = SequentialTransformer(transformers_list=[union_1], input_features=["value"])
seq_2 = SequentialTransformer(transformers_list=[date_features, date_lags], input_features=["date"])
union = UnionTransformer(transformers_list=[seq_1, seq_2])

pipeline = Pipeline(union, multivariate=False)

### <a id='toc1_1_3_'></a>[Trainer](#toc0_)

Currently, the available architectures are `DLinear` and `PatchTST`. However, adding your own architecture is quite simple if you follow the logic of the base model class.

In [6]:
# Configure the model parameters
model = DLinear_NN  # DLinear_NN or PatchTST
model_params = {"moving_avg": 7, "individual": False}

# Configure the validation parameters
validation = KFoldCrossValidator
validation_params = {
    "n_splits": 2,
}

#### Scheduler with num_steps != num_epochs

Since users don’t have direct access to len(dataset), Tsururu provides custom relative parameters inside `scheduler_params` to correct schedulers' parameters:

- `relative_steps_per_epoch`: defines a multiplier for iterations in one epoch. Used for parameters like steps_per_epoch in OneCycleLR.
  - $absolute\_steps\_per\_epoch = relative\_steps\_per\_epoch * (len(dataset) // batch\_size + 1)$

- `relative_steps`: defines a multiplier of the total number of iterations across all epochs. Used for parameters, such as T_max in CosineAnnealingLR.
    - $absolute\_steps = relative\_steps * n\_epochs * (len(dataset) // batch\_size + 1)$

In [7]:
# 1) Default Scheduler (no custom parameters needed):
scheduler = torch.optim.lr_scheduler.StepLR
scheduler_after_epoch = True
scheduler_params = {
    "step_size": 2,
}

# 2) OneCycleLR using relative_steps_per_epoch:
scheduler = torch.optim.lr_scheduler.OneCycleLR
scheduler_after_epoch = False
scheduler_params = {
    "relative_steps_per_epoch": ["steps_per_epoch"],
    "steps_per_epoch": 1.0,
    "epochs": 10,
    "max_lr": 0.01,
}

# 3) 
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR
scheduler_after_epoch = False
scheduler_params = {
    "relative_steps": ["T_max"],
    "T_max": 1.0,
}

In [8]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "best_by_metric": True,
    "save_to_dir": False,
    "n_epochs": 1,
    "scheduler": scheduler,
    "scheduler_after_epoch": scheduler_after_epoch,
    "scheduler_params": scheduler_params,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

### <a id='toc1_1_4_'></a>[Strategy](#toc0_)

In [9]:
horizon = 7
model_horizon = 7
history = 7

In [10]:
strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

In [11]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935
Updating learning rate to 0.001000.
Updating learning rate to 0.001000.
Updating learning rate to 0.000999.
Updating learning rate to 0.000998.
Updating learning rate to 0.000997.
Updating learning rate to 0.000996.
Updating learning rate to 0.000995.
Updating learning rate to 0.000993.
Updating learning rate to 0.000992.
Updating learning rate to 0.000990.
Updating learning rate to 0.000988.
Updating learning rate to 0.000985.
Updating learning rate to 0.000983.
Updating learning rate to 0.000980.
Updating learning rate to 0.000977.
Updating learning rate to 0.000974.
Updating learning rate to 0.000971.
Updating learning rate to 0.000967.
Updating learning rate to 0.000963.
Updating learning rate to 0.000959.
Updating learning rate to 0.000955.
Updating learning rate to 0.000951.
Updating learning rate to 0.000947.
Updating learning rate to 0.000942.
Updating learning rate to 0.000937.
Updating learning rate to 0.000932.
Updati

(14.50718879699707, <tsururu.strategies.mimo.MIMOStrategy at 0x15f017ca0>)

In [12]:
forecast_time, current_pred = strategy.predict(dataset)

freq: Day; period: 1
length of test dataset: 10


In [13]:
current_pred

,id,date,value
0,0,2022-09-27,3585.068604
1,0,2022-09-28,3585.391113
2,0,2022-09-29,3585.747314
3,0,2022-09-30,3586.063721
4,0,2022-10-01,3586.43042
...,...,...,...
65,9,2022-09-29,8631.006836
66,9,2022-09-30,8631.605469
67,9,2022-10-01,8632.235352
68,9,2022-10-02,8632.810547


Saving and loading checkpoints is an essential practice in training DL models. 

Let's explore how to save checkpoints to disk, what structure the saved files have, and how to restore the model from a checkpoint for either fine-tuning or inference.

# <a id='toc2_'></a>[Save and load checkpoints](#toc0_)

Let’s consider working with checkpoints using the Direct strategy as an example.

In [14]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "best_by_metric": True,
    # Let's enable save_to_dir (by the way, default value is True)
    "save_to_dir": True,
    "checkpoint_path": "checkpoints/",
    # Save checkpoints for 3 best model
    "save_k_best": 3,
    # Average checkpoints for the final model
    "average_snapshots": True,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

### <a id='toc2_1_1_'></a>[Save checkpoint](#toc0_)

In [15]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935


Epoch 1/10, cost time: 3.39s
train loss: 4760315.3496
Validation, Loss: 1037346.0000, Metric: -1037346.0000
val loss: 1037346.0000, val metric: -1037346.0000
Epoch 2/10, cost time: 3.21s
train loss: 696966.6131
Validation, Loss: 449721.6562, Metric: -449721.6562
val loss: 449721.6562, val metric: -449721.6562
Epoch 3/10, cost time: 2.99s
train loss: 267829.6388
Validation, Loss: 146554.3906, Metric: -146554.3906
val loss: 146554.3906, val metric: -146554.3906
Epoch 4/10, cost time: 3.00s
train loss: 78356.4891
Validation, Loss: 36455.9297, Metric: -36455.9297
val loss: 36455.9297, val metric: -36455.9297
Epoch 5/10, cost time: 3.10s
train loss: 17834.5238
Validation, Loss: 7310.2305, Metric: -7310.2305
val loss: 7310.2305, val metric: -7310.2305
Epoch 6/10, cost time: 3.04s
train loss: 3308.6349
Validation, Loss: 1244.0461, Metric: -1244.0461
val loss: 1244.0461, val metric: -1244.0461
Epoch 7/10, cost time: 3.02s
train loss: 592.8199
Validation, Loss: 270.2701, Metric: -270.2701
val l

(123.02275204658508, <tsururu.strategies.mimo.MIMOStrategy at 0x177e43160>)

## <a id='toc2_2_'></a>[Load checkpoint for finetune](#toc0_)

Once we have the saved checkpoints, we can continue training by passing the pretrained path and another checkpoint path to the trainer’s parameters. All other parameters remain the same.

In [16]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "best_by_metric": True,
    # Let's enable save_to_dir (by the way, default value is True)
    "save_to_dir": True,
    "pretrained_path": "checkpoints/",
    "checkpoint_path": "checkpoints_finetuned/",
    # Save checkpoints for 3 best model
    "save_k_best": 3,
    # Average checkpoints for the final model
    "average_snapshots": True,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

In [17]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935
Epoch 1/10, cost time: 3.12s
train loss: 18.0514
Validation, Loss: 12.2320, Metric: -12.2320
val loss: 12.2320, val metric: -12.2320
Epoch 2/10, cost time: 3.12s
train loss: 8.3653
Validation, Loss: 5.4261, Metric: -5.4261
val loss: 5.4261, val metric: -5.4261
Epoch 3/10, cost time: 3.09s
train loss: 3.6042
Validation, Loss: 2.2123, Metric: -2.2123
val loss: 2.2123, val metric: -2.2123
Epoch 4/10, cost time: 3.21s
train loss: 1.4245
Validation, Loss: 0.8411, Metric: -0.8411
val loss: 0.8411, val metric: -0.8411
Epoch 5/10, cost time: 3.14s
train loss: 0.5225
Validation, Loss: 0.2953, Metric: -0.2953
val loss: 0.2953, val metric: -0.2953
Epoch 6/10, cost time: 3.09s
train loss: 0.1785
Validation, Loss: 0.0985, Metric: -0.0985
val loss: 0.0985, val metric: -0.0985
Epoch 7/10, cost time: 3.15s
train loss: 0.0599
Validation, Loss: 0.0340, Metric: -0.0340
val loss: 0.0340, val metric: -0.0340
Epoch 8/10, cost time: 3.81s
train loss: 

(133.23628973960876, <tsururu.strategies.mimo.MIMOStrategy at 0x177e429b0>)

## <a id='toc2_3_'></a>[Load checkpoint for inference](#toc0_)

In [18]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "n_epochs": 1,
    "pretrained_path": "checkpoints_finetuned/",
    # Average checkpoints for the final model
    "average_snapshots": True,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

In [19]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935
Epoch 1/1, cost time: 3.27s
train loss: 0.0088
Validation, Loss: 0.0073, Metric: -0.0073
val loss: 0.0073, val metric: -0.0073
Fold 0. Score: -0.007292281370609999
length of train dataset: 4935
length of val dataset: 4935
Epoch 1/1, cost time: 3.91s
train loss: 0.0724
Validation, Loss: 0.0694, Metric: -0.0694
val loss: 0.0694, val metric: -0.0694
Fold 1. Score: -0.06937484443187714
Mean score: -0.0383
Std: 0.031


(13.822022914886475, <tsururu.strategies.mimo.MIMOStrategy at 0x15f017850>)

In [20]:
forecast_time, current_pred = strategy.predict(dataset)

freq: Day; period: 1
length of test dataset: 10


In [21]:
current_pred

,id,date,value
0,0,2022-09-27,1999.862915
1,0,2022-09-28,2000.86499
2,0,2022-09-29,2001.868896
3,0,2022-09-30,2002.870972
4,0,2022-10-01,2003.874146
...,...,...,...
65,9,2022-09-29,11001.836914
66,9,2022-09-30,11002.838867
67,9,2022-10-01,11003.842773
68,9,2022-10-02,11004.847656
